# Playground

In [1]:
%reload_ext autoreload
%autoreload 3

import torch
import torch.nn.functional as F
from torchinfo import summary
import pytorch_lightning as pl
from src.b2bnet import B2BNetModel, Autoencoder, OtkaDataModule, OtkaTimeDimSplit
from pytorch_lightning.loggers import TensorBoardLogger

## LSTM Autoencoder

In [2]:
# Experiment

segment_size = 120 * 3  # 3sec
batch_size = 256
n_features = 59
hidden_size = 59
max_epochs = 200

datamodule = OtkaTimeDimSplit(segment_size=segment_size, batch_size=batch_size)

model = Autoencoder(n_features=n_features, hidden_size=hidden_size, n_subjects=51)

trainer = pl.Trainer(max_epochs=max_epochs,accelerator='cpu', log_every_n_steps=1, deterministic='warn')

trainer.fit(model, datamodule=datamodule)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/homebrew/Caskroom/miniforge/base/envs/b2bnet/lib/python3.10/site-packages/pytorch_lightning/trainer/setup.py:201: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.
  rank_zero_warn(

  | Name           | Type   | Params
------------------------------------------
0 | encoder        | LSTM   | 28.3 K
1 | relu           | ReLU   | 0     
2 | decoder        | LSTM   | 28.3 K
3 | fc_decoder     | Linear | 3.5 K 
4 | b2b_decoder    | LSTM   | 28.3 K
5 | fc_b2b_decoder | Linear | 3.5 K 
6 | cls            | Linear | 120   
7 | clf_subject    | Linear | 3.1 K 
------------------------------------------
95.2 K    Trainable params
0         Non-trainable params
95.2 K    Total params
0.381     Total estimated model params size (MB)
Global seed set to 42


Sanity Checking: 0it [00:00, ?it/s]

/opt/homebrew/Caskroom/miniforge/base/envs/b2bnet/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/opt/homebrew/Caskroom/miniforge/base/envs/b2bnet/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=200` reached.


## GNN
TODO: This code receives multiple timeseries, transforms them into graphs, and then applies a GNN to them. The graph embeddings are then used for downstream tasks.